In [5]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from collections import defaultdict
import itertools
import time
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

######### Generate bigrams from a word
def generate_bigrams(word):
    return [word[i:i+2] for i in range(len(word) - 1)]

####### Process bigrams (sorted + remove duplicates)
def process_bigrams(bigrams):
    unique_bigrams = sorted(set(bigrams))
    return unique_bigrams[:5]

###### Prepare the dataset
def prepare_dataset(dictionary):
    bigram_dict = defaultdict(list)
    for word in dictionary:
        bigrams = generate_bigrams(word)
        processed_bigrams = process_bigrams(bigrams)
        bigram_dict[tuple(processed_bigrams)].append(word)
    return bigram_dict

# Implement the my_fit() method that takes a dictionary as a list of words and returns a trained ML model.
def my_fit(dictionary):
    bigram_dict = prepare_dataset(dictionary)
    
    X = []  # Store feature vectors
    y = []  # Store words
    
    # bigram_to_index is a dictionary that maps each unique bigram to a unique index.
    bigram_to_index = {bigram: idx for idx, bigram in enumerate(sorted(set(itertools.chain(*bigram_dict.keys()))))}
    
    # Keeping track of all possible words for each set of bigrams
    bigram_to_words = defaultdict(list)
    
    for bigrams, words in bigram_dict.items():
        feature_vector = [0] * len(bigram_to_index)
        for bigram in bigrams:
            feature_vector[bigram_to_index[bigram]] = 1
        X.append(feature_vector)
        
        # Associate this set of bigrams with all possible words
        for word in words:
            bigram_to_words[tuple(feature_vector)].append(word)
    
    X = np.array(X)
    # Use the first word of each bigram set for training
    y = np.array([words[0] for words in bigram_dict.values()])
    
    # Train the decision tree model
    model = DecisionTreeClassifier(max_depth=10)
    model.fit(X, y)
    
    return model, bigram_to_index, bigram_to_words

# Implement the my_predict() method that takes your learnt model and a tuple of bigrams
# sorted in ascending order, and returns a list of guesses. The my_predict() must return a
# list even if it is making only a single guess.
def my_predict(model, bigram_to_index, bigram_to_words, bigrams):
    processed_bigrams = process_bigrams(bigrams)
    feature_vector = [0] * len(bigram_to_index)
    for bigram in processed_bigrams:
        if bigram in bigram_to_index:
            feature_vector[bigram_to_index[bigram]] = 1
    
    feature_tuple = tuple(feature_vector)
    
    # Retrieve all possible words that match this set of bigrams
    if feature_tuple in bigram_to_words:
        predictions = bigram_to_words[feature_tuple]
    else:
        # If no match in bigram_to_words, use model prediction and look up the closest matches
        prediction = model.predict([feature_vector])[0]
        prediction_bigrams = tuple(process_bigrams(generate_bigrams(prediction)))
        predictions = bigram_dict[prediction_bigrams] if prediction_bigrams in bigram_dict else [prediction]
    
    return predictions[:5]

# Load the dictionary
with open('dict_secret.txt', 'r') as file:
    dictionary = [line.strip() for line in file]

n_trials = 5
t_train = 0
m_size = 0
t_test = 0
prec = 0

def get_bigrams(word, lim=None):
    # Get all bigrams
    bg = map(''.join, list(zip(word, word[1:])))
    # Remove duplicates and sort them
    bg = sorted(set(bg))
    # Make them into an immutable tuple and retain only the first few
    return tuple(bg)[:lim]

lim_bg = 5
lim_out = 5

for t in range(n_trials):
    tic = time.perf_counter()
    model, bigram_to_index, bigram_to_words = my_fit(dictionary)
    toc = time.perf_counter()
    t_train += toc - tic

    with open(f"model_dump_{t}.pkl", "wb") as outfile:
        pickle.dump((model, bigram_to_index, bigram_to_words), outfile, protocol=pickle.HIGHEST_PROTOCOL)

    m_size += os.path.getsize(f"model_dump_{t}.pkl")

    tic = time.perf_counter()

    for (i, word) in enumerate(dictionary):
        bg = get_bigrams(word, lim=lim_bg)
        guess_list = my_predict(model, bigram_to_index, bigram_to_words, bg)

        # Do not send long guess lists -- they will result in lower marks
        guess_len = len(guess_list)
        # Ignore all but the first 5 guesses
        guess_list = guess_list[:lim_out]

        # Notice that if 10 guesses are made, one of which is correct,
        # score goes up by 1/10 even though only first 5 guesses are considered
        # Thus, it is never beneficial to send more than 5 guesses
        if word in guess_list:
            prec += 1 / guess_len

    toc = time.perf_counter()
    t_test += toc - tic

t_train /= n_trials
m_size /= n_trials
t_test /= n_trials
prec /= (n_trials * len(dictionary))

print(f"Training Time: {t_train:.6f} seconds")
print(f"Model Size: {m_size:.6f} bytes")
print(f"Prediction Time: {t_test:.6f} seconds")
print(f"Precision: {prec:.4f}")

# Example predictions
test_bigrams = [
    ('al','io','na','on','op'),
    ('ar','ca','rr','ry'),
    ('ar','re','ea'),
    ('ci','en','ff','fi','ic'),
    ('be','de','es','id','si')
]

# Predict for each set of bigrams and print the results
for bigrams in test_bigrams:
    print(f"Bigrams: {bigrams}")
    predictions = my_predict(model, bigram_to_index, bigram_to_words, bigrams)
    print(f"Predictions: {predictions}\n")


Training Time: 0.688947 seconds
Model Size: 9412287.000000 bytes
Prediction Time: 0.070713 seconds
Precision: 0.9750
Bigrams: ('al', 'io', 'na', 'on', 'op')
Predictions: ['optional', 'proportional']

Bigrams: ('ar', 'ca', 'rr', 'ry')
Predictions: ['carry']

Bigrams: ('ar', 're', 'ea')
Predictions: ['area', 'rear']

Bigrams: ('ci', 'en', 'ff', 'fi', 'ic')
Predictions: ['insufficient', 'sufficient', 'sufficiently']

Bigrams: ('be', 'de', 'es', 'id', 'si')
Predictions: ['beside', 'besides']

